# Test/Dev Neural Network Architecture

In [19]:
from __future__ import print_function

import sys
import os
import time

import numpy as np

import chb

## Load Subject Data

In [15]:
subject = chb.CHBsubj()
subject.load_meta('chb22')
subject.load_data(exthd=False)

Loading: /Users/adamcellon/Drive/senior/thesis/data/chb22.npz
Done: 66.427477 seconds elapsed.


In [37]:
eeg = subject[0].get_rec()

print(eeg.shape)
print('Deleting empty channels...')
eeg = np.delete(eeg, [4,9,12,17,22], 0)

print(eeg.shape)

(28, 921600)
Deleting empty channels...
(23, 921600)
